# Granularity
Standardise the data such that they have the same granularity.  

 * Weekly from Week #17 2012
 * Locale : Singapore

Import modules:-

 *  pandas as pd
 *  numpy as np
 *  os
 *  glob
 *  re
 * from datetime import date

### Set directory paths:-

In [ ]:
raw_path = "../../data/1_raw/"
out_path = "../../data/2_interim/"
clean_path = "../../data/3_clean/"

if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

### Compile weather data into Weekly, Singapore average format

In [ ]:
weather_files = glob.glob(raw_path+'weather/*.csv')
len(weather_files)

#### Create DataFrame from file:

In [ ]:
dfWeather = pd.read_csv(weather_files.pop(), encoding='latin_1')
for f in weather_files:
    df = pd.read_csv(f, encoding='latin_1')
    dfWeather = dfWeather.append(df)

Use DataFrame describe and head to inspect data:

#### Remove unecessary columns:

In [ ]:
# inspect columns

Use drop() to remove four columns

#### Rename column headers into usable labels, i.e. standard ASCII UTF-8 characters.

#### Deal with missing / invalid data

#### Deal with improbable data:

In [ ]:
dfWeather['Minimum Temperature'] = dfWeather['Minimum Temperature'].apply(lambda x: np.nan if x==0 else x)

In [ ]:
dfWeather.describe()

#### Get week number:

In [ ]:
def get_weeknum(row):
    isocal = date(int(row['Year']), int(row['Month']), int(row['Day'])).isocalendar()
    weeknum = str(isocal[0]) + '-W' + "%02d"%isocal[1] 
    return weeknum

In [ ]:
dfWeather['Y-Week'] = dfWeather.apply(lambda row: get_weeknum(row), axis=1)

In [ ]:
dfWeather.head()

#### Recalculate data by week number:

In [ ]:
group_dict = {'Minimum Temperature':['min'], 
              'Daily Rainfall Total':['mean'], 
              'Mean Temperature':['mean'], 
              'Mean Wind Speed':['mean'], 
              'Maximum Temperature':['max'], 
              'Max Wind Speed':['max']}

In [ ]:
groupedWeather = dfWeather.groupby(['Y-Week']).agg(group_dict)


In [ ]:
groupedWeather.head()

In [ ]:
groupedWeather.columns

#### Rename columns:

In [ ]:
groupedWeather.columns = [col[0] for col in groupedWeather.columns]

In [ ]:
groupedWeather.columns

In [ ]:
groupedWeather.head()

In [ ]:
groupedWeather = groupedWeather.rename(index=str, columns={'Daily Rainfall Total':'Mean Daily Rainfall Total'})

#### Save to file:
Save file to interim folder

# Dengue and Malaria data
Read the infectious disease data:

In [ ]:
dfDisease = 

Keep only the Malaria, Dengue Fever and Dengue Haemorrhagic Fever data

### Pivoting
Use pivot() to get data by week:

Combine Dengue Fever and Dengue Haemorrhagic Fever into one column

And remove unnecessary columns

#### Save to file

# Combine Data
Combine the two data sets into one DataFrame

#### Remove invalid rows with no data:

### Reformat indexing

In [ ]:
YearWeek = [[int(x) for x in item.split('-W')] for item in dfData.index]


In [ ]:
dfYearWeek = pd.DataFrame.from_records(YearWeek, index = dfData.index, columns = ['Year', 'Week'])

In [ ]:
dfDataYW = pd.concat([dfData,dfYearWeek], axis=1)

In [ ]:
dfDataYW = dfDataYW.set_index(['Year','Week'])

In [ ]:
dfDataYW.head()

#### Save to file
Save combined data frame into clean folder